In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
HUGGINGFACE_MODEL_PATH = "beomi/KcELECTRA-small-v2022"
batch_size = 8
num_epochs = 2

In [3]:
! pip install transformers
! pip install datasets

In [5]:
# #  TPU 초기화
# import tensorflow as tf
# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)


In [6]:
# strategy = tf.distribute.TPUStrategy(resolver)

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

In [8]:
data = pd.read_csv('/content/drive/MyDrive/train.csv')

In [9]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3}

num_labels = len(labels)
data_ = data.copy()

data_['label'] = data_['class'].apply(lambda x: labels[x])

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=len(labels))


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-small-v2022 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
len(labels)

4

In [12]:
import pyarrow as pa
from datasets import Dataset

data_ = Dataset(pa.Table.from_pandas(data_))

In [13]:
data_

Dataset({
    features: ['idx', 'class', 'conversation', 'label'],
    num_rows: 3950
})

In [14]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )
encoded_data = data_.map(transform, batched=True)

Map:   0%|          | 0/3950 [00:00<?, ? examples/s]

In [15]:
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [16]:
pip install transformers[torch]

In [17]:
metric = load_metric("f1")

def compute_metrics(eval_pred):
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/'

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01
)

<ipython-input-17-44201a0ac865>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [18]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.076778,0.746835
2,1.270700,0.914612,0.815190


TrainOutput(global_step=890, training_loss=1.144595474071717, metrics={'train_runtime': 7886.8298, 'train_samples_per_second': 0.902, 'train_steps_per_second': 0.113, 'total_flos': 209184873062400.0, 'train_loss': 1.144595474071717, 'epoch': 2.0})

In [19]:


# Load Test-set
test_file_path = '/content/drive/MyDrive/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()


In [20]:
test_X = Dataset(pa.Table.from_pandas(test_data))

In [21]:
def test_transform(data):
  return tokenizer(
      data['text'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_test_data = test_X.map(test_transform, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [22]:
prediction = trainer.predict(encoded_test_data)

In [23]:
len(prediction.predictions)
pred_label = []

for i in range(len(prediction.predictions)):
    pred_label.append((str)(np.argmax(prediction.predictions[i])).zfill(2))

pred_label[:10]

['01', '02', '02', '03', '00', '00', '01', '01', '00', '02']

In [28]:
test_submission = test_data.assign(CLASS=pred_label)
test_submission = test_submission.rename(columns={'CLASS':'class'})

In [30]:
test_submission.drop(['text'], axis=1, inplace=True)

In [31]:
test_submission

,class
t_000,01
t_001,02
t_002,02
t_004,03
t_005,00
...,...
t_495,02
t_496,02
t_497,01
t_498,02


In [34]:
import pandas as pd


submission_file_path = '/content/drive/MyDrive/KcELECTRA-small_2epo_submission.csv'
test_submission.to_csv(submission_file_path, index=True)

